# Source code for predicting Ames House Prices

In [3]:
# Importing the libraries
import pandas as pd
import numpy as np
import re

In [4]:
# Loading/reading in the data
DatasetDF = pd.read_csv(r'../datasets/train.csv')
DatasetDF

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


## Performing data quality checks

In [123]:
# Checking the data types
print('==========Data types of the columns=========')
print(DatasetDF.dtypes)

# Checking the cardinality of categorical data
print('\n=========Cardinality of the categorical columns==========')
CategoricalCols = DatasetDF.select_dtypes(object)    #Get the columns of categorical data type
for col in list(CategoricalCols.columns):     #Create a list with column names and loop through the list
    print(f'{col} {CategoricalCols[col].unique()}')     #Display the unique records in each column

# Checking for missing values
print('\n=======Columns with missing values and the number of missing values=========')
MissingValueCols = DatasetDF.columns[DatasetDF.isna().any()].tolist()     #Get the columns with missing values and convert them to a list
for col in MissingValueCols:
    print(f'{col}: {DatasetDF[col].isna().sum()}')

# Checking the formatting of categorical data
print('\n=======Checking the formatting of categorical data(checking for white spaces)=======\n')
for col in list(CategoricalCols.columns):    #checking for extra white spaces
    if (CategoricalCols[col].str.strip() != CategoricalCols[col]).sum() > 0:
        print(f'{col}: {(CategoricalCols[col].str.strip() != CategoricalCols[col]).sum()}')

# Checking for duplicate records
print('\n=========Number of duplicate records based on Id==========')
DuplicateRecords = DatasetDF.duplicated(subset=['Id']).sum()
print(f'Duplicates: {DuplicateRecords}')    #Decided to check based on Id only,since all other rows can repeat 

==========Data types of the columns=========
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

=========Cardinality of the categorical columns==========
MSZoning ['RL' 'RM' 'C (all)' 'FV' 'RH']
Street ['Pave' 'Grvl']
Alley [nan 'Grvl' 'Pave']
LotShape ['Reg' 'IR1' 'IR2' 'IR3']
LandContour ['Lvl' 'Bnk' 'Low' 'HLS']
Utilities ['AllPub' 'NoSeWa']
LotConfig ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
LandSlope ['Gtl' 'Mod' 'Sev']
Neighborhood ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1 ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRA

## Data Cleaning and Transformation

In [24]:
DatasetDFCopy = DatasetDF.copy()     #Performing data cleaning on a copy of the original dataset
cleanDataDF = (DatasetDFCopy.drop(columns=['LotFrontage','LotArea','Alley','LandContour','LotConfig','LandSlope','Neighborhood','Condition1','Condition2',
                                            'BldgType','YearBuilt','YearRemodAdd','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','ExterQual',
                                           'BsmtQual','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','Heating',
                                           'HeatingQC','CentralAir','Electrical','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath',
                                           'FullBath','HalfBath', 'BedroomAbvGr', 'KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','Fireplaces','FireplaceQu','GarageType',
                                           'GarageYrBlt','GarageFinish','GarageArea','GarageQual','GarageCond','PavedDrive','WoodDeckSF','OpenPorchSF','EnclosedPorch',
                                           '3SsnPorch','ScreenPorch','PoolArea','PoolQC','MiscFeature','MiscVal','MoSold','YrSold']))

# Cleaning the BsmtCond and Fence columns
cleanDataDF['BsmtCond'] = cleanDataDF['BsmtCond'].fillna('None')
cleanDataDF['Fence'] = cleanDataDF['Fence'].fillna('None')
# print(cleanDataDF)